# Look at the results of capsid BLAST and determine percent identity cutoffs, samples, and viruses that make sense to continue exploring for viral genome recovery

## Notebook setup

In [6]:
library(dplyr, warn.conflicts = F) 
library(readr)
library(purrr)
library(tidyr)
library(stringr)
library(janitor, warn.conflicts = F)

In [7]:
setwd("..")

## Functions for notebook

In [8]:
# from data frame, write a generic fasta file
write_fasta <- function(data, filename){
  fasta_lines <- c()
  for (rowNum in 1:nrow(data)){
    fasta_lines <- c(fasta_lines, as.character(paste(">", data[rowNum, "read_name"], sep = "")))
    fasta_lines <- c(fasta_lines, as.character(data[rowNum, "seq"]))
  }
  
  file_connection <- file(filename)
  writeLines(fasta_lines, file_connection)
  close(file_connection)
}

In [9]:
# from dimaond blastx tsv results, write a fasta file that preserves paired end information
filter_and_write_fasta <- function(df, filter_threshold, sseqid_i, output_folder, output_suffix){
  df %>%
    filter(pident > filter_threshold) %>%
    filter(sseqid == sseqid_i) %>%
    select(run, read_number, full_qseq, full_qseq_mate) %>%
    pivot_longer(cols = -starts_with("r"), names_to = "read_pair", values_to = "seq") %>%
    mutate(read_pair = ifelse(read_pair == "full_qseq", 1, 2),
           read_name = paste0(run, ".", read_number, "/", read_pair)) %>%
    select(read_name, seq, run) %>%
    distinct() %>%
    group_by(run) %>%
    group_walk(~ write_fasta(., filename = paste0(output_folder, .y$run, "-", sseqid_i, output_suffix)))
}

## Read in diamond blastx against capsid db results

In [10]:
files <- Sys.glob("outputs/capsid_blast_raw_lca/*tsv")
capsid_blast <-  files %>%
  map_dfr(read_tsv, col_types = "cdddccdddddcccccdcc") %>%
  separate(qseqid, into = c("run", "read_number", "read_pair"), sep = "\\.", remove = F)
# write_tsv(capsid_blast, "outputs/capsid_blast_raw_lca_combined.tsv.gz")

In [11]:
capsid_blast_paired_reads_mapped <- capsid_blast %>%
  group_by(run, read_number) %>%
  tally() %>%
  filter(n >= 2) %>%
  mutate(run_read_number = paste0(run, read_number, sep = "_"))

In [12]:
# filter to reads where both pairs were hits
capsid_blast_filtered <- capsid_blast %>%
  mutate(run_read_number = paste0(run, read_number, sep = "_")) %>%
  filter(run_read_number %in% capsid_blast_paired_reads_mapped$run_read_number)
# write_tsv(capsid_blast_filtered, "outputs/capsid_blast_raw_lca_combined_filtered_to_pe.tsv.gz")

## What viruses do we detect with confidence?

In [21]:
other_viruses <- capsid_blast_filtered %>%  
  filter(pident >= 90) %>%
  group_by(lca_lineage_named, sseqid) %>%
  tally() %>%
  arrange(desc(n))

other_viruses_num_bioprojects <- capsid_blast_filtered %>%
  left_join(metadata, by = "run") %>%
  select(lca_lineage_named, sseqid, bio_project) %>%
  distinct() %>%
  group_by(lca_lineage_named, sseqid) %>%
  tally() %>%
  arrange(desc(n)) %>%
  select(lca_lineage_named, sseqid, num_bio_projects = n) %>%
  mutate(possible_num_bio_projects = length(unique(metadata$bio_project)))

other_viruses_num_runs <- capsid_blast_filtered %>%
  left_join(metadata, by = 'run') %>%
  select(lca_lineage_named, sseqid, run) %>%
  distinct() %>%
  group_by(lca_lineage_named, sseqid) %>%
  tally() %>%
  arrange(desc(n)) %>%
  select(lca_lineage_named, sseqid, num_runs = n) %>%
  mutate(possible_num_runs = length(unique(files)))

other_viruses <- other_viruses %>%
  left_join(other_viruses_num_runs) %>%
  left_join(other_viruses_num_bioprojects)
write_tsv(other_viruses, "virus_abundances_pident90.tsv")

Joining with `by = join_by(lca_lineage_named, sseqid)`
Joining with `by = join_by(lca_lineage_named, sseqid)`


In [24]:
other_viruses

lca_lineage_named,sseqid,n,num_runs,possible_num_runs,num_bio_projects,possible_num_bio_projects
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>
Viruses;Bamfordvirae;Preplasmiviricota;Tectiliviricetes;Rowavirales;Adenoviridae;Mastadenovirus;Human mastadenovirus C;unclassified Human mastadenovirus C subspecies/strain,10515.AP_000172.1,4244,128,342,1,11
Viruses;Shotokuvirae;Cossaviricota;Papovaviricetes;Sepolyvirales;Polyomaviridae;Deltapolyomavirus;Deltapolyomavirus decihominis;unclassified Deltapolyomavirus decihominis subspecies/strain,1203539.YP_006495786.1,575,2,342,1,11
Viruses;Heunggongvirae;Uroviricota;Caudoviricetes;unclassified Caudoviricetes order;Demerecviridae;Novosibvirus;Novosibvirus PM135;unclassified Novosibvirus PM135 subspecies/strain,2048008.YP_009620573.1,490,154,342,7,11
Viruses;Shotokuvirae;Cossaviricota;Papovaviricetes;Sepolyvirales;Polyomaviridae;Deltapolyomavirus;Deltapolyomavirus decihominis;unclassified Deltapolyomavirus decihominis subspecies/strain,1203539.YP_006495788.1,435,1,342,1,11
Viruses;Shotokuvirae;Cossaviricota;Papovaviricetes;Sepolyvirales;Polyomaviridae;Deltapolyomavirus;Deltapolyomavirus undecihominis;unclassified Deltapolyomavirus undecihominis subspecies/strain,1277649.YP_007354884.1,108,3,342,2,11
Viruses;Shotokuvirae;Cossaviricota;Papovaviricetes;Sepolyvirales;Polyomaviridae;Deltapolyomavirus;Deltapolyomavirus undecihominis;unclassified Deltapolyomavirus undecihominis subspecies/strain,1277649.YP_007354882.1,94,2,342,1,11
Viruses;Heunggongvirae;Uroviricota;Caudoviricetes;unclassified Caudoviricetes order;unclassified Caudoviricetes family;unclassified Caudoviricetes genus;unclassified Caudoviricetes species;unclassified Caudoviricetes subspecies/strain,1147146.YP_007112546.1,93,3,342,1,11
Viruses;Shotokuvirae;Cossaviricota;Papovaviricetes;Sepolyvirales;Polyomaviridae;Alphapolyomavirus;Alphapolyomavirus cardiodermae;unclassified Alphapolyomavirus cardiodermae subspecies/strain,1891718.YP_007346963.1,65,156,342,6,11
Viruses;Heunggongvirae;Uroviricota;Caudoviricetes;unclassified Caudoviricetes order;unclassified Caudoviricetes family;unclassified Caudoviricetes genus;unclassified Caudoviricetes species;unclassified Caudoviricetes subspecies/strain,1147146.YP_007112551.1,64,1,342,1,11


### Firgure out what percent identity is the most reliable for working directly from capsid blast results -- minimizing false positives

In [ ]:
capsid_blast_pident <- capsid_blast_filtered %>%
  filter(pident > 67) # theres a breakpoint in the pident.

capsid_blast_pident_tally <- capsid_blast_pident %>%
  group_by(lca_lineage_named, sseqid) %>%
  tally() %>%
  arrange(desc(n))

tmp80 <- capsid_blast_filtered %>%
  filter(pident > 80)

tmp90 <- capsid_blast_filtered %>%
  filter(pident > 90) 

tmp80 %>%
  filter(!sseqid %in% tmp90$sseqid) %>%
  group_by(lca_lineage_named, sseqid) %>%
  tally() %>%
  arrange(desc(n))

# for each of thresholds 67, 80, and 90, pick a capsid that has a lot of hits.
# write them to FASTA to assemble & BLAST to see if there are any real viruses.

# boca virus for 67:
filter_and_write_fasta(df = capsid_blast_filtered, 
                       filter_threshold = 67, 
                       sseqid = "2053079.YP_009552697.1", 
                       output_folder = "outputs/capsid_blast_explore/",
                       output_suffix = "_boca_reads.fasta")

# 80:
filter_and_write_fasta(df = capsid_blast_filtered, 
                       filter_threshold = 80, 
                       sseqid = "1922325.YP_009508741.1", 
                       output_folder = "outputs/capsid_blast_explore/",
                       output_suffix = "_deltapap_reads.fasta")

# 80/90:
filter_and_write_fasta(df = capsid_blast_filtered, 
                       filter_threshold = 90, 
                       sseqid = "1203539.YP_006495786.1", 
                       output_folder = "outputs/capsid_blast_explore/",
                       output_suffix = "deltapolyoma_reads.fasta")
# 90:
filter_and_write_fasta(df = capsid_blast_filtered, 
                       filter_threshold = 90, 
                       sseqid = "10515.AP_000172.1", 
                       output_folder = "outputs/capsid_blast_explore/",
                       output_suffix = "mastadeno_reads.fasta")

I think I tried to assemble these and they largely didn't assemble -- only the deltapolyoma did.
Because there were so few reads, I BLASTed each read against NCBI nt using remote BLAST from the command line:
```
for infile in *fasta
do
  bn=$(basename $infile .fasta)
  blastn -db nt -query {input} -out ${bn}.tsv -remote -max_target_seqs 100 -outfmt 6
done
```
I then looked at the results (see the `blast_has_virus` notebook).
From these analyses, I saw that under 90\% identity, there were a lot of hits to human and vectors/clones.

I conclude 90\% is a reasonable cutoff to use; everything else was off target.
Because we were not interested in finding phages, I then grab non caudo (phage) viruses that are in more than one bioproject to increase the chances that what we were seeing were real viruses that end up in the brain and are not due to contamination.

In [26]:
candidates <- other_viruses %>% filter(num_bio_projects > 1) %>% 
  filter(!grepl(pattern = "Caudo", x = lca_lineage_named))

capsid_blast_filtered %>%
  filter(pident > 90) %>%
  filter(sseqid %in% candidates$sseqid) %>%
  group_by(run, sseqid) %>%
  tally() %>%
  arrange(desc(n)) 

for(sseqid in candidates$sseqid){
  #print(sseqid)
  filter_and_write_fasta(df = capsid_blast_filtered,
                         filter_threshold = 90,
                         sseqid_i = sseqid,
                         output_folder = "outputs/capsid_blast_pident90/",
                         output_suffix = ".fasta")
}
#cat(candidates$sseqid)

run,sseqid,n
<chr>,<chr>,<int>
SRR1778915,1277649.YP_007354884.1,92
SRR23515743,1891718.YP_007346963.1,48
SRR1779424,1891767.NP_043126.1,35
SRR1779200,1277649.YP_007354884.1,16
SRR1779422,1891767.NP_043126.1,6
SRR1779396,465447.YP_001837096.1,4
SRR1779415,1891767.NP_043126.1,4
SRR8750456,10617.NP_040895.1,4
SRR8750473,10617.NP_040895.1,4


## Mimivirus had the most hits for one sample. Look at it

In [30]:
mimi_filtered <- capsid_blast_filtered %>%
  filter(sseqid == "2487768.AYV81705.1") 
head(mimi_filtered)

# these look like repetitive elements and not that interesting, skipping even though it's the most common

qseqid,run,read_number,read_pair,qstart,qend,qlen,qstrand,sseqid,sstart,⋯,evalue,cigar,qseq_translated,full_qseq,full_qseq_mate,taxid,lca_taxid,lca_lineage_named,lca_lineage_taxid,run_read_number
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,⋯,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
ERR2262601.66311333.1,ERR2262601,66311333,1,146,27,150,-,2487768.AYV81705.1,345,⋯,2.48e-04,40M,GLGLGSGLGLGLRVGLRLGLGSGLGVKLGLGLGHGMGVGS,CCCAGGCCCAATTTCAGGCCCAGTCCAGATCCCACGCCCATTCCGTGGCCCAATCCCAGGCCCAATTTCACGCCCAGCCCCGATCCCAGGCCCAATCTCAGGCCCACTCTCAGGCCCAAACCCAGGCCCGATCCCAGACCCAAACCCCGC,GGGCTCGGGCCTGGGGTTGGGCCTGGGCTTGGGCCTGAGATTGGGCCTGGGACCGGTCCTGGGGTTGGGACTGGGCGTGGGCTTGGGCGGGGGCATGGGCGGGGGGCGGGGACTGGGGGGGGGTCCGGGCCGGGGCGGGCGGAGGAGGCT,2487768,2487768,Viruses;Bamfordvirae;Nucleocytoviricota;Megaviricetes;Imitervirales;Mimiviridae;unclassified Mimiviridae genus;Harvfovirus sp.;unclassified Harvfovirus sp. subspecies/strain,10239;2732005;2732007;2732523;2732554;549779;;2487768;,ERR226260166311333_
ERR2262601.66311333.2,ERR2262601,66311333,2,2,139,150,+,2487768.AYV81705.1,328,⋯,3.40e-04,19M1I26M,GSGLGLGLGLGLRLGLGPVLGLGLGVGLGGGMGGGRGLGGGPGRGG,GGGCTCGGGCCTGGGGTTGGGCCTGGGCTTGGGCCTGAGATTGGGCCTGGGACCGGTCCTGGGGTTGGGACTGGGCGTGGGCTTGGGCGGGGGCATGGGCGGGGGGCGGGGACTGGGGGGGGGTCCGGGCCGGGGCGGGCGGAGGAGGCT,CCCAGGCCCAATTTCAGGCCCAGTCCAGATCCCACGCCCATTCCGTGGCCCAATCCCAGGCCCAATTTCACGCCCAGCCCCGATCCCAGGCCCAATCTCAGGCCCACTCTCAGGCCCAAACCCAGGCCCGATCCCAGACCCAAACCCCGC,2487768,2487768,Viruses;Bamfordvirae;Nucleocytoviricota;Megaviricetes;Imitervirales;Mimiviridae;unclassified Mimiviridae genus;Harvfovirus sp.;unclassified Harvfovirus sp. subspecies/strain,10239;2732005;2732007;2732523;2732554;549779;;2487768;,ERR226260166311333_
ERR2262601.81533231.2,ERR2262601,81533231,2,149,3,150,-,2487768.AYV81705.1,320,⋯,5.11e-05,25M1D24M,GVGAGLGWGLGLGLGLGLGSGLGLGLGLGLRLGLGSGLGSGLRLGLGSG,NGGCCCGATCCCAGGCCCAATCTCAGTCCCGATCCCAGGCCCGATCCCAGGCCCAATCTCAGGCCCAACCCCAGGCCCAGACCCAGGCCCGATCCCAGGCCCAATCCCAGGCCCAAGCCCAGGCCCCACCCCAGGCCCGCACCCACGCCC,NGATGGGACCTGGGCCTTGGAGTCTGCCTGGGCCTGGGATCTGGGCTGGGCTGGGAATCGGGCCTATAATTGGGCCTGGGCCTGAGATTGGGCCTGGGATTGGGCCTGGGATCAGGCCTGGGTTCGGGCCTGAGATTGGGCCTGGGATCG,2487768,2487768,Viruses;Bamfordvirae;Nucleocytoviricota;Megaviricetes;Imitervirales;Mimiviridae;unclassified Mimiviridae genus;Harvfovirus sp.;unclassified Harvfovirus sp. subspecies/strain,10239;2732005;2732007;2732523;2732554;549779;;2487768;,ERR226260181533231_
ERR2262601.262314214.2,ERR2262601,262314214,2,149,6,150,-,2487768.AYV81705.1,318,⋯,1.32e-04,27M1I20M,GLGSGLGLKLGLGLGHGMGLGSGLGLELGLGLRLGSGLGSDLGLELGL,AATTCCAGGCCCAATTCCAGGCCCAGGTCAGATCCCAAGCCCGATCCCAGCCGCAATCCCAGGCCCAATTCCAGGCCCAGTCCAGATCCCAGGCCCATTCCGTGGCCCAATCCCAGGCCCAATTTCAGGCCCAGCCCCGATCCCAGGCCC,GGATCGGGACTGAGATTGGGCCTGGGATCGGGCCTGAGATTGGGCCTGGGATCGGGCCTGGGTTTGGGCCTGGGATCGGGCCTGGGTTTGGGCCTGAGAATGGGCCTGAGATTGGGCCTGGGATCGGGGCTGGGCCTGAAATTGGGCCTG,2487768,2487768,Viruses;Bamfordvirae;Nucleocytoviricota;Megaviricetes;Imitervirales;Mimiviridae;unclassified Mimiviridae genus;Harvfovirus sp.;unclassified Harvfovirus sp. subspecies/strain,10239;2732005;2732007;2732523;2732554;549779;;2487768;,ERR2262601262314214_
ERR2262601.292737445.2,ERR2262601,292737445,2,136,5,149,-,2487768.AYV81705.1,318,⋯,5.11e-05,27M3D17M,GLGSGLGLELGLGVGHGLGLGSGLGLKLGLGLGLGSGLGSELGL,ATTCCAGGCCCAGCTCAGATCCCAAGCCCGATCCCAGCCCCAATCCCAGGCCCAATTTCAGGCCCAGTCCAGATCCCAGGCCCAGTCCGTGGCCCACTCCCAGGCCCAATTCCAGGCCCAGCCCCGATCCCAGGCCCAAGCCCAGGCCC,GGGCCTGGGATCGGGCCTGGGATCGGGACTGAGATTGGGCCTGGGATCGGGCCTGAGATTGGGCCTGGGATCGGGCCTGGGTTTGGGCCTGGGATCGGGCCTGGGTTTGGGCCTGAGAATGGGCCTGAGATTGGGCCTGGGATCGGGGCT,2487768,2487768,Viruses;Bamfordvirae;Nucleocytoviricota;Megaviricetes;Imitervirales;Mimiviridae;unclassified Mimiviridae genus;Harvfovirus sp.;unclassified Harvfovirus sp. subspecies/strain,10239;2732005;2732007;2732523;2732554;549779;;2487768;,ERR2262601292737445_
ERR2262601.334105009.2,ERR2262601,334105009,2,148,5,149,-,2487768.AYV81705.1,326,⋯,9.61e-05,19M9D29M,GLGLGWGLGPGVGSGLGL

## SessionInfo

In [32]:
sessionInfo()

R version 4.2.3 (2023-03-15)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/taylorreiter/miniconda3/envs/sandbox/lib/libopenblasp-r0.3.21.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] janitor_2.2.0 stringr_1.5.0 tidyr_1.3.0   purrr_1.0.1   readr_2.1.4  
[6] dplyr_1.1.2  

loaded via a namespace (and not attached):
 [1] pillar_1.9.0     compiler_4.2.3   base64enc_0.1-3  tools_4.2.3     
 [5] bit_4.0.5        digest_0.6.31    uuid_1.1-0       jsonlite_1.8.4  
 [9] lubridate_1.9.2  evaluate_0.20    lifecycle_1.0.3  tibble_3.2.1    
[13] timechange_0.2.0 pkgconfig_2.0.3  rlang_1.1.0      IRdisplay_1.1   
[17] cli_3.6.1        parallel_4.2.3   IRkernel_1.3.2   fastmap_1.1.1   
[21] withr_2.5.0      repr_1.1.6       generics_0.1.3   vctrs_0.6.1     